In [1]:
import pickle
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf

Using TensorFlow backend.


In [2]:
with open('df_sector_UTD.pickle','rb') as f:
    df_sector_UTD = pickle.load(f)
with open('certificated_stock_dic.pickle','rb') as f:
    certificated_stock_dic = pickle.load(f)
with open('X_scaler_dic.pickle','rb' ) as f:
    X_scaler_dic = pickle.load(f)


# 스케일링 하기

In [3]:
from sklearn.preprocessing import MinMaxScaler
scale_cols = ['거래량','시가','고가', '저가','종가']

X_UTD_scaled_dic = {}
for sector_name, stock_list in certificated_stock_dic.items():
    X_UTD_scaled_dic[sector_name] = {}
    for stock_code in stock_list:
        X_scaler = X_scaler_dic[sector_name][stock_code]
        X_UTD_scaled = X_scaler.transform(df_sector_UTD[sector_name][stock_code][scale_cols])
        X_UTD_scaled_dic[sector_name][stock_code] = pd.DataFrame(X_UTD_scaled,index = df_sector_UTD[sector_name][stock_code].index,columns=scale_cols)
        

In [4]:
X_UTD_scaled_dic

{'Food': {'004370':            거래량        시가        고가        저가        종가
  2999  0.040367  0.086131  0.082111  0.081967  0.079646
  2998  0.027292  0.072993  0.068915  0.083607  0.079646
  2997  0.045317  0.071533  0.082111  0.085246  0.087021
  2996  0.014893  0.080292  0.085044  0.098361  0.094395
  2995  0.022189  0.086131  0.087977  0.098361  0.092920
  ...        ...       ...       ...       ...       ...
  4     0.023890  0.246715  0.246334  0.280328  0.255162
  3     0.029492  0.246715  0.249267  0.272131  0.246313
  2     0.069618  0.242336  0.239003  0.265574  0.249263
  1     0.109163  0.233577  0.231672  0.252459  0.231563
  0     0.093930  0.216058  0.241935  0.250820  0.253687
  
  [3000 rows x 5 columns]},
 'Clothing': {},
 'Chemical': {},
 'Medicine': {},
 'Non_Metal': {'003410':            거래량        시가        고가        저가        종가
  0     0.004713  0.043014  0.038611  0.045311  0.041754
  1     0.005005  0.043014  0.038004  0.045038  0.038997
  2     0.015182  0.03

In [5]:
#윈도우 사이즈
WINDOW_SIZE = 10
# X에 대한 윈도우 함수 설정값
X_size = WINDOW_SIZE
X_shift = 1
X_stride = 1
# y에 대한 윈도우 함수 설정값
y_size = 1
y_shift = 1
y_stride = 1

batch_size = 32

# 2. 모델 불러오기

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [7]:
model_dic ={}
for sector_name, stock_list in certificated_stock_dic.items():
    model_dic[sector_name] = {}
    for stock_code in stock_list:
        model_dic[sector_name][stock_code] = Sequential([
            # 1차원 feature map 생성
            Conv1D(filters=32, kernel_size=5,
                   padding="causal",
                   activation="relu",
                   #feature의 개수 3개
                   input_shape=[WINDOW_SIZE, 5]),
            # LSTM
            LSTM(16, activation='tanh'),
            Dense(16, activation="relu"),
            Dense(3,activation='softmax'),
        ])

In [8]:
with open('filename_dic.pickle', 'rb') as f:
    filename_dic = pickle.load(f)

In [9]:
filename_dic

{'Food': {'097950': 'MC\\Food\\097950ckeckpointer.ckpt',
  '271560': 'MC\\Food\\271560ckeckpointer.ckpt',
  '000080': 'MC\\Food\\000080ckeckpointer.ckpt',
  '004370': 'MC\\Food\\004370ckeckpointer.ckpt',
  '005300': 'MC\\Food\\005300ckeckpointer.ckpt'},
 'Clothing': {'093050': 'MC\\Clothing\\093050ckeckpointer.ckpt',
  '020000': 'MC\\Clothing\\020000ckeckpointer.ckpt',
  '105630': 'MC\\Clothing\\105630ckeckpointer.ckpt',
  '001070': 'MC\\Clothing\\001070ckeckpointer.ckpt'},
 'Chemical': {'051910': 'MC\\Chemical\\051910ckeckpointer.ckpt',
  '096770': 'MC\\Chemical\\096770ckeckpointer.ckpt',
  '010950': 'MC\\Chemical\\010950ckeckpointer.ckpt',
  '051900': 'MC\\Chemical\\051900ckeckpointer.ckpt',
  '090430': 'MC\\Chemical\\090430ckeckpointer.ckpt'},
 'Medicine': {'207940': 'MC\\Medicine\\207940ckeckpointer.ckpt',
  '068270': 'MC\\Medicine\\068270ckeckpointer.ckpt',
  '000100': 'MC\\Medicine\\000100ckeckpointer.ckpt',
  '128940': 'MC\\Medicine\\128940ckeckpointer.ckpt'},
 'Non_Metal': {'00

In [10]:
for_linux = {}
for sector_name, sector_name_dic in filename_dic.items():
    for_linux[sector_name] = {}
    for stock_code, filename in sector_name_dic.items():
        linux_filename = filename.replace('\\','/')
        for_linux[sector_name][stock_code] = linux_filename

In [11]:
for_linux

{'Food': {'097950': 'MC/Food/097950ckeckpointer.ckpt',
  '271560': 'MC/Food/271560ckeckpointer.ckpt',
  '000080': 'MC/Food/000080ckeckpointer.ckpt',
  '004370': 'MC/Food/004370ckeckpointer.ckpt',
  '005300': 'MC/Food/005300ckeckpointer.ckpt'},
 'Clothing': {'093050': 'MC/Clothing/093050ckeckpointer.ckpt',
  '020000': 'MC/Clothing/020000ckeckpointer.ckpt',
  '105630': 'MC/Clothing/105630ckeckpointer.ckpt',
  '001070': 'MC/Clothing/001070ckeckpointer.ckpt'},
 'Chemical': {'051910': 'MC/Chemical/051910ckeckpointer.ckpt',
  '096770': 'MC/Chemical/096770ckeckpointer.ckpt',
  '010950': 'MC/Chemical/010950ckeckpointer.ckpt',
  '051900': 'MC/Chemical/051900ckeckpointer.ckpt',
  '090430': 'MC/Chemical/090430ckeckpointer.ckpt'},
 'Medicine': {'207940': 'MC/Medicine/207940ckeckpointer.ckpt',
  '068270': 'MC/Medicine/068270ckeckpointer.ckpt',
  '000100': 'MC/Medicine/000100ckeckpointer.ckpt',
  '128940': 'MC/Medicine/128940ckeckpointer.ckpt'},
 'Non_Metal': {'003670': 'MC/Non_Metal/003670ckeckpoin

In [12]:
for sector_name, stock_list in certificated_stock_dic.items():
    for stock_code in stock_list:
        if not stock_list:
            model_dic[sector_name][stock_code].load_weights(linux_filename[sector_name][stock_code])

# 존재 항목별 tomorrow 예측하기

In [13]:
from datetime import date,datetime, timedelta
today = date.today()
tomorrow = today+timedelta(days=1, hours = 9)
tomorrow = tomorrow.isoformat()
tomorrow = tomorrow.replace('-','')[2:]
set_d = tomorrow
set_d


'220809'

In [14]:
with open('sector_name_kr.pickle','rb' ) as f:
    sector_name_kr = pickle.load(f)

In [15]:
sector_name_kr

{'Food': ['CJ제일제당', '오리온', '하이트진로', '농심', '롯데칠성'],
 'Clothing': ['LF', '한섬', '한세실업', '대한방직'],
 'Chemical': ['LG화학', 'SK이노베이션', 'S-Oil', 'LG생활건강', '아모레퍼시픽'],
 'Medicine': ['삼성바이오로직스', '셀트리온', '유한양행', '한미약품'],
 'Non_Metal': ['포스코케미칼', '쌍용C&E', '아이에스동서'],
 'Metal': ['POSCO홀딩스', '고려아연', '현대제철', 'KG스틸', '동국제강'],
 'Machine': ['두산에너빌리티', '한온시스템', '두산밥캣', '씨에스윈드'],
 'Electronic': ['삼성전자', 'SK하이닉스', '삼성SDI', 'LG전자'],
 'Construction': ['현대건설', 'GS건설', '대우건설', '한전KPS'],
 'Transport': ['HMM', '대한항공', '현대글로비스', '한진칼', '팬오션'],
 'Distribution': ['삼성물산', '롯데쇼핑', 'BGF리테일', '이마트', '신세계'],
 'Power': ['한국전력', '한국가스공사', '서울가스'],
 'Tele': ['SK텔레콤', 'KT', 'LG유플러스'],
 'Finance': ['LG', '삼성화재', '미래에셋증권'],
 'Brokerage': ['NH투자증권', '삼성증권', '메리츠증권', '키움증권'],
 'Insurer': ['삼성생명', 'DB손해보험', '메리츠화재', '현대해상'],
 'Service': ['NAVER', '카카오', '삼성에스디에스', '엔씨소프트'],
 'Manufacturer': ['현대차', '기아', '현대모비스', 'KT&G', '삼성전기']}

In [23]:
with open('stock_acc_dic.pickle','rb') as f:
    stock_acc_dic = pickle.load(f)

In [29]:
pred_dic = {}
for sector_name, sector_model in model_dic.items():
    pred_dic[sector_name] = {}
    for stock_code, model in sector_model.items():
        pred_dic[sector_name][stock_code] = []
        data = tf.constant(np.expand_dims(np.array(X_UTD_scaled_dic[sector_name][stock_code].tail(10))[:10],axis=0))
        result = model.predict(data)
        print('-----------{0}-----------'.format(stock_code))
        acc = f'accuracy : {np.around(stock_acc_dic[sector_name][stock_code]*100,2)}'
        up = f'상향할 확률 : {np.around(result[0][0]*100,1)}'
        down = f'하향할 확률 : {np.around(result[0][1]*100,1)}'
        sideways = f'횡보할 확률 : {np.around(result[0][2]*100,1)}'
        pred_dic[sector_name][stock_code].append(acc)
        pred_dic[sector_name][stock_code].append(up)
        pred_dic[sector_name][stock_code].append(down)
        pred_dic[sector_name][stock_code].append(sideways)
        print(acc)
        print(up)
        print(down)
        print(sideways)

-----------004370-----------
accuracy : 57.82
상향할 확률 : 32.7
하향할 확률 : 33.4
횡보할 확률 : 33.9
-----------003410-----------
accuracy : 82.48
상향할 확률 : 30.6
하향할 확률 : 32.1
횡보할 확률 : 37.3
-----------010130-----------
accuracy : 55.44
상향할 확률 : 32.5
하향할 확률 : 32.4
횡보할 확률 : 35.0
-----------047040-----------
accuracy : 71.26
상향할 확률 : 33.4
하향할 확률 : 33.5
횡보할 확률 : 33.1
-----------028670-----------
accuracy : 65.31
상향할 확률 : 33.3
하향할 확률 : 33.3
횡보할 확률 : 33.4
-----------028260-----------
accuracy : 55.65
상향할 확률 : 33.6
하향할 확률 : 32.5
횡보할 확률 : 34.0
-----------032640-----------
accuracy : 58.16
상향할 확률 : 32.9
하향할 확률 : 30.7
횡보할 확률 : 36.4
-----------006800-----------
accuracy : 73.13
상향할 확률 : 33.3
하향할 확률 : 33.3
횡보할 확률 : 33.3
-----------005940-----------
accuracy : 61.22
상향할 확률 : 32.8
하향할 확률 : 34.3
횡보할 확률 : 32.9
-----------008560-----------
accuracy : 88.61
상향할 확률 : 31.7
하향할 확률 : 39.9
횡보할 확률 : 28.4
-----------012330-----------
accuracy : 56.63
상향할 확률 : 33.3
하향할 확률 : 31.6
횡보할 확률 : 35.1


In [30]:
with open('pred_dic.pickle','wb' ) as f:
    pickle.dump(pred_dic,f)